In [1]:
import ollama
import requests
import json

In [2]:
model = "llama3.2:1b"

#### Task 1: Interact with deployed LLM via python 


**Objective:**

Explore different techniques to interact with the deployed LLM.

**Task Description:**

1. Use Request libaray (HTTP Client) and send a POST request to interact with the LLM: [How To](https://requests.readthedocs.io/en/latest/user/quickstart/#make-a-request)

In [3]:
# Simple HTTP Request via requests

# Define the URL of the deployed LLM ( this port is forwarded from the docker container to the host system)
url = "http://localhost:11434/api/generate"

# Define the prompt as json
body_json = {
    "model": model,
    "prompt": "Describe Generative AI in two sentences."
}

# ADD HERE YOUR CODE
# HINT: Send the POST request using the json body
response = requests.post(url, json=body_json)

# Check if the request was successful
if response.status_code == 200:
    # Process the response
    response_text = response.text

    # Convert each line to json
    response_lines = response_text.splitlines()
    response_json = [json.loads(line) for line in response_lines]
    for line in response_json:
        # Print the response. No line break
        print(line["response"], end="", flush=True)
else:
    print("Error:", response.status_code)


Generative AI refers to a class of artificial intelligence algorithms that can create new data or content, such as text, images, music, and videos, that is similar in style or quality to existing examples in the same domain. By combining machine learning with generative models, Generative AI enables computers to generate novel output that can be used for various purposes, including creative applications, product design, and data augmentation.

**Task Description:**

2. Use Ollama python library to interact with the LLM: [How To](https://pypi.org/project/ollama/)

- First use method ``ollama.chat(...)``
- First use method ``ollama.chat(...)`` with ``stream=True``

In [4]:
# API Call via ollama

# Definiere die Nachricht(en), die an das Modell gesendet werden sollen
messages = [
    {
        'role': 'user',
        'content': 'Was ist der schnellste Landtier?',
    },
]
# ADD HERE YOUR CODE

response = ollama.chat(model='llama3.2:1b', messages=messages)


print(response["message"]["content"])

Der schnellste Landtier auf unserem Planeten ist der Löwenteuerer (Cercopithecus ascanius). Er erreicht eine Geschwindigkeit von etwa 70-80 Kilometern pro Stunde. Der Löwenteuerer ist ein relativ großer und starkes Tier, das in den Wäldern Afrikas lebt.


In [5]:
# Streaming API Call via ollama
messages = [
    {
        'role': 'user',
        'content': 'Schreibe einen kurzen Absatz über die Vorzüge des Programmierens.',
    },
]
# Response streaming can be enabled by setting stream=True, 
# modifying function calls to return a Python generator where each part is an object in the stream.

# ADD HERE YOUR CODE

stream = ollama.chat(model=model, messages=messages,  stream=True)

for chunk in stream:
    print(chunk["message"]["content"], end="", flush=True)

Das Programmieren ist eine faszinierende Welt, in der Menschen ihre Fähigkeiten entwickeln und ihre Probleme lösen können. Durch das Programmieren können Menschen verschiedene Dinge erreichen, wie zum Beispiel komplexe Aufgaben automatisch ausführen oder effizientere Lösungen für Herausforderungen finden. Es ist auch ein großartiger Weg, um Technologie zu lernen und sich selbst fortzubilden.

#### Task 2: Experimenting with Prompt Techniques

**Objective:**

Objective: Explore different prompt techniques (Zero Shot, One Shot, and Few Shot) by sending different types of prompts to the LLM.

![image](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*QSpK--jqPiUU_OHuZvtUWA.png)

**Task Description:**

1. Create three prompts for a sentiment analysis task: a Zero Shot prompt, a One Shot prompt, and a Few Shot prompt. Use the examples from the table above.
2. Send these prompts to the LLM and observe the differences in the responses.
3. Compare and discuss the responses.

In [6]:
import ollama
# ADD HERE YOUR PROMPTS

zero_shot_prompt = """Klassifiziere das Sentiment des folgenden Textes als 'Positiv' oder 'Negativ':
Text: Der Film war unglaublich langweilig und die Handlung war verwirrend.
Sentiment:"""

one_shot_prompt = """Klassifiziere das Sentiment von Texten als 'Positiv' oder 'Negativ'.

Beispiel:
Text: Das Essen im Restaurant war fantastisch und der Service war schnell.
Sentiment: Positiv

Text: Der Film war unglaublich langweilig und die Handlung war verwirrend.
Sentiment:"""

few_shot_prompt = """Klassifiziere das Sentiment von Texten als 'Positiv' oder 'Negativ'.

Beispiele:
Text: Das Essen im Restaurant war fantastisch und der Service war schnell.
Sentiment: Positiv

Text: Die Lieferzeit war sehr lang, was enttäuschend war.
Sentiment: Negativ

Text: Die neue Software ist effizient und einfach zu bedienen.
Sentiment: Positiv

Text: Der Film war unglaublich langweilig und die Handlung war verwirrend.
Sentiment:"""

# Stream the responses and print them
for idx, prompt in enumerate([zero_shot_prompt, one_shot_prompt, few_shot_prompt]):
    prompt_type = ["Zero-Shot", "One-Shot", "Few-Shot"][idx]
    print(f"\n--- {prompt_type} Prompt ---\n")
    print(f"User Prompt:\n{prompt}\n")
    
    stream = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    
    print("Model Output:")
    for chunk in stream:
        print(chunk["message"]["content"], end="", flush=True)
    print("\n-----------------------------\n")



--- Zero-Shot Prompt ---

User Prompt:
Klassifiziere das Sentiment des folgenden Textes als 'Positiv' oder 'Negativ':
Text: Der Film war unglaublich langweilig und die Handlung war verwirrend.
Sentiment:

Model Output:
Das Sentiment des gegebenen Textes ist negativ. Die Worte "unglaublich" und "langweilig" sowie das Verwendung von Worten wie "verwirrend" deuteten darauf hin, dass der Film als sehr schlechtmäßig und anstößig wahrgenommen wurde.
-----------------------------


--- One-Shot Prompt ---

User Prompt:
Klassifiziere das Sentiment von Texten als 'Positiv' oder 'Negativ'.

Beispiel:
Text: Das Essen im Restaurant war fantastisch und der Service war schnell.
Sentiment: Positiv

Text: Der Film war unglaublich langweilig und die Handlung war verwirrend.
Sentiment:

Model Output:
Ich würde das erste Beispiel als "Positiv" bezeichnen, da das Text beschreibt, wie das Essen im Restaurant war, was eine positive Erfahrung erwartet werden könnte.

Das zweite Beispiel ist etwas anderes. D

#### Task 3: Prompt Refinement and Optimization

**Objective:** 

Refine a prompt to improve the clarity and quality of the LLM's response.

**Task Description:**

- Start with a basic prompt asking the LLM to summarize a paragraph.
- Refine the prompt by adding specific instructions to improve the summary's quality. (Example: define how long the summary should be, define on which to focus in the summary)

In [7]:
import ollama
# Original prompt
original_prompt = "Summarize the following paragraph: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries."

# ADD HERE YOUR PROMPT
refined_prompt = "Summarize the following paragraph in exactly one sentence. The summary must clearly define Generative AI and list its primary application areas: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries."

# Stream the responses and print them
for idx, prompt in enumerate([original_prompt, refined_prompt]):
    prompt_type = ["Original Prompt", "Refined Prompt"][idx]
    print(f"\n--- {prompt_type} ---\n")
    print(f"User Prompt:\n{prompt}\n")
    
    stream = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )
    
    print("Model Output:")
    for chunk in stream:
        print(chunk["message"]["content"], end="", flush=True)
    print("\n-----------------------------\n")



--- Original Prompt ---

User Prompt:
Summarize the following paragraph: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative industries.

Model Output:
Generative AI is an area of AI that uses patterns learned from data to create new content such as text, images, and music. This field has various applications in creative industries and is becoming increasingly used.
-----------------------------


--- Refined Prompt ---

User Prompt:
Summarize the following paragraph in exactly one sentence. The summary must clearly define Generative AI and list its primary application areas: Generative AI is a field of artificial intelligence focused on creating new content based on patterns learned from existing data. It has applications in text, image, and music generation, and is increasingly being used in creative indust

#### [Optional] Task 4: Structured Prompting with Roles (Pirate Theme)

**Objective:**

Learn how to use structured prompts that combine role assignment, clear instructions, and examples to improve the output of language models. In this task, you will guide the AI to respond as a pirate who is also an expert in machine learning.

**Instructions:**

- Role Assignment: In your prompt, specify the role of the AI as a Machine Learning Expert who speaks like a pirate.

- Instruction: Clearly state what you want the AI to explain or discuss in pirate language.

- Examples: Provide examples to guide the AI in using pirate lingo while explaining technical concepts.

In [8]:
import ollama
# Combined Techniques Prompt with Pirate Theme

structured_prompt = """Ahoy there, me hearty! Listen close to yer old Captain, for I be both a salty sea dog and a **Machine Learning Expert**.

**Role Assignment:** Ye shall respond in the manner of a wise, seafaring pirate captain. All yer explanations must be riddled with pirate lingo and nautical terms (e.g., 'Ahoy!', 'me hearty', 'Shiver me timbers').

**Instruction:** Explain the machine learning concept of **Gradient Descent**. Make sure ye cover what it is used for and why it is important for training models.

**Example of yer speak:**
User: Explain a loss function.
AI Pirate: A loss function, aye, is like the treasure map that tells ye how far yer ship (yer model) has strayed from the buried gold (the true answer). The smaller the loss, the closer ye be to the bounty!

Now, explain **Gradient Descent**, ye scallywag!
"""

# Stream the response and print it
print("=== User Prompt ===")
print(structured_prompt)

stream = ollama.chat(
    model=model,
    messages=[{"role": "user", "content": structured_prompt}],
    stream=True,
)

print("\n=== Model Output ===")
for chunk in stream:
    print(chunk["message"]["content"], end="", flush=True)
print("\n")


=== User Prompt ===
Ahoy there, me hearty! Listen close to yer old Captain, for I be both a salty sea dog and a **Machine Learning Expert**.

**Role Assignment:** Ye shall respond in the manner of a wise, seafaring pirate captain. All yer explanations must be riddled with pirate lingo and nautical terms (e.g., 'Ahoy!', 'me hearty', 'Shiver me timbers').

**Instruction:** Explain the machine learning concept of **Gradient Descent**. Make sure ye cover what it is used for and why it is important for training models.

**Example of yer speak:**
User: Explain a loss function.
AI Pirate: A loss function, aye, is like the treasure map that tells ye how far yer ship (yer model) has strayed from the buried gold (the true answer). The smaller the loss, the closer ye be to the bounty!

Now, explain **Gradient Descent**, ye scallywag!


=== Model Output ===
Arrrr, me hearty machine learnin' expert! Yer lookin' fer a lesson on Gradient Descent, eh? Alright then, settle yerself down with a pint o' g